In [9]:
import numpy as np
import math

In [2]:
a = np.loadtxt('../dataset/scan/pose/0.txt')

In [7]:
def logarithm_so3(R_mat):
    SE3_ANGLE_APPROX_THRESH_PI = 1.0
    SE3_ANGLE_APPROX_THRESH_ZERO = 0.03    
    OneOver6 = 1.0/6.0
    OneOver7 = 1.0/7.0
    OneOver9 = 1.0/9.0
    def A_approximation(theta2):
        return 1.0 - theta2 * OneOver6 * (1.0 - 0.05 * theta2 * (1.0 - theta2 * OneOver6 * OneOver7))

    trR = float(R_mat.trace().clip(-1.0, 3.0))
    theta = math.acos((trR - 1.0) * 0.5)
    ln_Rvec = np.empty(3, dtype=np.float32)
    if theta > np.pi - SE3_ANGLE_APPROX_THRESH_PI:
        R_diag = R_mat.diagonal()
        a = int(R_diag.argmax())
        b = (a+1) % 3
        c = (a+2) % 3
        s = math.sqrt(1.0 + R_diag[a]-R_diag[b]-R_diag[c])
        ln_Rvec[a] = s * 0.5
        ln_Rvec[b] = (R_mat[b,a] + R_mat[a,b]) * 0.5 / s
        ln_Rvec[c] = (R_mat[c,a] + R_mat[a,c]) * 0.5 / s
        vNorm = ln_Rvec.dot(ln_Rvec)
        if vNorm != 0:
            ln_Rvec /= vNorm
        wNorm = (R_mat[c,b] - R_mat[b,c]) * 0.5 / s
        ln_Rvec *= 2.0 * math.atan(vNorm/wNorm)
    else:
        theta2 = 0
        A = 0
        if theta > SE3_ANGLE_APPROX_THRESH_ZERO:
            theta2 = theta * theta
            A = math.sin(theta) / theta
        else:
            theta2 = R_mat[0,1] * R_mat[0,1] + R_mat[0,2] * R_mat[0,2] + R_mat[1,2] * R_mat[1,2]
            A = A_approximation(theta2)
        ln_R = (R_mat - R_mat.T) * 0.5 / A
        ln_Rvec[0] = 0.5 * (ln_R[2,1] - ln_R[1,2])
        ln_Rvec[1] = 0.5 * (ln_R[0,2] - ln_R[2,0])
        ln_Rvec[2] = 0.5 * (ln_R[1,0] - ln_R[0,1])
    return ln_Rvec    

def get_inplane_rotation(R_mat):
    rvec = logarithm_so3(R_mat)
    zvec = np.array([0,0,1], dtype=np.float32)
    rot_val = np.dot(rvec, zvec) # radian
    return rot_val

In [6]:
a[:3,:3]

array([[ 0.93874 ,  0.188266, -0.288657],
       [ 0.344322, -0.547558,  0.762642],
       [-0.014477, -0.815314, -0.578839]])

In [10]:
get_inplane_rotation(a[:3,:3])

0.22946559